# обработка новостей

In [2]:
import pandas as pd
import numpy as np

In [11]:
train = pd.read_csv('/kaggle/input/finam-data2-merge/train_all.csv')
train

,open,close,high,low,volume,begin,ticker
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT
1,81.72,82.10,83.98,80.26,18502950,2020-06-22,AFLT
2,82.04,81.20,82.48,80.40,16848930,2020-06-23,AFLT
3,79.78,80.58,80.80,78.22,21559860,2020-06-25,AFLT
4,80.50,79.38,81.44,78.76,14677280,2020-06-26,AFLT
...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,2025-09-04,VTBR
25938,75.85,75.50,76.13,74.94,40701093,2025-09-05,VTBR
25939,75.59,75.64,75.71,75.35,1681425,2025-09-06,VTBR
25940,75.69,75.68,75.85,75.50,2186520,2025-09-07,VTBR


In [12]:
train_new = pd.read_csv('/kaggle/input/finam-data2-merge/train_all_new.csv')
train_new

,publish_date,title,publication
0,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смот...,Тенденции в отрасли. Ключевые российские нефте...
1,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на вну...,Тенденции в отрасли. Ключевые российские стале...
2,2020-01-03 10:13:10,Контракты на поставку газа в Белоруссию и тран...,"Председатель правления ""Газпрома"" Алексей Милл..."
3,2020-01-03 13:26:29,"ПАО ""ФосАгро"" -Внеочередное общее собрание",Дата и время ВОСА – 24.01.2020 23:59:59 Дата ...
4,2020-01-03 13:44:03,"ПАО ""ММК"" - Внеочередное общее собрание- ИТОГИ",Дата и время собрания - 27.12.2019 00:00:00 Д...
...,...,...,...
27450,2025-09-05 19:20:50,Invesco Large Cap Value ETF испытывает большой...,"Сегодня, рассматривая изменения в количестве а..."
27451,2025-09-05 21:20:00,Финал недели оказался благоприятным для рынка ...,Финал недели на российском рынке прошел на опт...
27452,2025-09-06 09:34:00,Вторая партия газа с «Арктик СПГ 2» доставлена...,Второй танкер со сжиженным природным газом с п...
27453,2025-09-06 10:29:00,Индекс МосБиржи удерживается выше 2900 пунктов...,Российский рынок демонстрирует минимальные изм...


In [13]:
import pandas as pd
import numpy as np
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm_asyncio
import time
from typing import List, Tuple

# --- Конфигурация ---
OPENROUTER_API_KEY = "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"

# Параметры для защиты API
MAX_CONCURRENT_REQUESTS = 200  # Количество одновременных запросов было 100 если сломается то со 100 не ломалось
REQUESTS_PER_MINUTE = 1000     # Лимит запросов в минуту
BATCH_SIZE = 100             # Размер батча для обработки
RETRY_ATTEMPTS = 3           # Количество попыток при ошибке
RETRY_DELAY = 2              # Задержка между попытками (секунды)

# Инициализация клиента
client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

# --- Семафор для контроля параллельности ---
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

# --- Контроль rate limit ---
class RateLimiter:
    def __init__(self, max_per_minute):
        self.max_per_minute = max_per_minute
        self.requests = []
    
    async def wait_if_needed(self):
        now = time.time()
        # Удаляем запросы старше минуты
        self.requests = [req_time for req_time in self.requests if now - req_time < 60]
        
        if len(self.requests) >= self.max_per_minute:
            sleep_time = 60 - (now - self.requests[0]) + 0.1
            if sleep_time > 0:
                await asyncio.sleep(sleep_time)
                self.requests = []
        
        self.requests.append(time.time())

rate_limiter = RateLimiter(REQUESTS_PER_MINUTE)

# --- Асинхронная функция классификации ---
async def get_ticker_async(title: str, text: str, tickers_str: str, 
                          valid_tickers: list, attempt: int = 0) -> str:
    """
    Асинхронная классификация новости с retry логикой
    """
    async with semaphore:
        await rate_limiter.wait_if_needed()
        
        system_prompt = (
            "You are an expert analyst of the Russian stock market with deep knowledge of publicly traded companies. "
            "Your task is to identify which company a news article primarily focuses on by selecting the correct ticker from a provided list. "
            "You must be precise and distinguish between primary subjects and secondary mentions."
        )
        
        user_prompt = f"""
Analyze the news article below and identify the PRIMARY company it discusses.

**Available tickers:** {tickers_str}

**NEWS ARTICLE:**
Title: "{title}"
Body: "{text}"

**ANALYSIS RULES:**
1. Identify the MAIN subject of the article - the company that is the primary focus
2. If multiple companies are mentioned, select the one that is the central topic
3. Ignore companies mentioned as examples, comparisons, or minor participants
4. If the news is about general market trends, macroeconomics, regulations, or doesn't specifically focus on any company from the list, return 'None'
5. If a company is mentioned through its subsidiaries, divisions, or projects, still return its ticker if it's in the list
6. News about mergers/acquisitions should return the ticker of the acquiring company or the main subject
7. Analyst recommendations, earnings reports, dividend announcements should return the ticker of the company being discussed

**EXAMPLES:**
Input: "Сбербанк увеличил чистую прибыль на 15% в первом квартале"
Output: SBER

Return ONLY the ticker symbol or 'None'. No explanations, no additional text, no punctuation.

**Your answer:**"""
        
        try:
            response = await client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=10,
                timeout=30.0
            )
            
            ticker = response.choices[0].message.content.strip()
            ticker = ticker.replace('.', '').replace(',', '').upper()
            
            if ticker in valid_tickers or ticker == 'NONE':
                return ticker
            else:
                return 'None'
                
        except Exception as e:
            if attempt < RETRY_ATTEMPTS:
                await asyncio.sleep(RETRY_DELAY * (attempt + 1))
                return await get_ticker_async(title, text, tickers_str, 
                                             valid_tickers, attempt + 1)
            else:
                print(f"Ошибка после {RETRY_ATTEMPTS} попыток: {str(e)}")
                return 'None'

# --- Функция для обработки батча ---
async def process_batch(batch_data: List[Tuple], tickers_str: str, 
                       valid_tickers: list) -> List[str]:
    """
    Обработка батча новостей
    """
    tasks = []
    for idx, title, text in batch_data:
        task = get_ticker_async(title, text, tickers_str, valid_tickers)
        tasks.append(task)
    
    results = await tqdm_asyncio.gather(*tasks, desc=f"Обработка батча")
    return results

# --- Основная функция ---
async def classify_news_async(news_df: pd.DataFrame, target_tickers: np.ndarray) -> pd.DataFrame:
    """
    Главная функция для классификации всех новостей
    """
    tickers_str = ", ".join(sorted(target_tickers))
    valid_tickers_list = list(target_tickers)
    
    # Подготовка данных
    batch_data = [
        (idx, row['title'], row['publication'])
        for idx, row in news_df.iterrows()
    ]
    
    # Разбиваем на батчи
    all_results = []
    for i in range(0, len(batch_data), BATCH_SIZE):
        batch = batch_data[i:i + BATCH_SIZE]
        print(f"\nОбработка батча {i//BATCH_SIZE + 1}/{(len(batch_data)-1)//BATCH_SIZE + 1}")
        
        batch_results = await process_batch(batch, tickers_str, valid_tickers_list)
        all_results.extend(batch_results)
        
        # Небольшая пауза между батчами
        if i + BATCH_SIZE < len(batch_data):
            await asyncio.sleep(1)
    
    # Добавляем результаты в датафрейм
    news_df['ticker'] = all_results
    return news_df

# --- Запуск ---
def main(news_df: pd.DataFrame, df: pd.DataFrame):
    """
    Обертка для запуска асинхронной обработки
    """
    # Получаем уникальные тикеры
    target_tickers = np.unique(df['ticker'])
    
    print(f"Найдено {len(target_tickers)} уникальных тикеров")
    print(f"Всего новостей для классификации: {len(news_df)}")
    print(f"Параметры обработки:")
    print(f"  - Максимум одновременных запросов: {MAX_CONCURRENT_REQUESTS}")
    print(f"  - Лимит запросов в минуту: {REQUESTS_PER_MINUTE}")
    print(f"  - Размер батча: {BATCH_SIZE}")
    print(f"  - Попыток при ошибке: {RETRY_ATTEMPTS}")
    
    start_time = time.time()
    
    # Запускаем асинхронную обработку
    result_df = asyncio.run(classify_news_async(news_df.copy(), target_tickers))
    
    elapsed_time = time.time() - start_time
    print(f"\n✓ Обработка завершена за {elapsed_time/60:.2f} минут")
    print(f"✓ Обработано {len(result_df)} новостей")
    print(f"✓ Классифицировано: {(result_df['ticker'] != 'None').sum()} новостей")
    print(f"✓ Не удалось классифицировать: {(result_df['ticker'] == 'None').sum()} новостей")
    
    return result_df



In [14]:
import nest_asyncio
nest_asyncio.apply()
result_df = main(train_new, train)


Найдено 19 уникальных тикеров
Всего новостей для классификации: 27455
Параметры обработки:
  - Максимум одновременных запросов: 200
  - Лимит запросов в минуту: 1000
  - Размер батча: 100
  - Попыток при ошибке: 3

Обработка батча 1/275


Обработка батча: 100%|██████████| 100/100 [00:02<00:00, 42.45it/s]



Обработка батча 2/275


Обработка батча: 100%|██████████| 100/100 [00:04<00:00, 23.94it/s]



Обработка батча 3/275


...

Обработка батча: 100%|██████████| 100/100 [00:30<00:00,  3.30it/s]

Ошибка после 3 попыток: Error code: 403 - {'error': {'message': 'Key limit exceeded. Manage it using https://openrouter.ai/settings/keys', 'code': 403}}



Обработка батча 92/275


Обработка батча:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
result_df

In [ ]:
result_df.to_csv('new_title.csv', index = False)

In [ ]:
import pandas as pd
import numpy as np


train_new = pd.read_csv('new_title.csv')


train = pd.read_csv('candles.csv')


In [ ]:

def merge_candles_and_news(candles_df, news_df, dataset_name='train'):
    """
    Объединяет данные свечей (цены) с новостями по тикеру и дате
    """
    # Копируем данные
    df_candles = candles_df.copy()
    df_news = news_df.copy()
    
    # 1. Преобразование дат
    df_candles['begin'] = pd.to_datetime(df_candles['begin'])
    df_candles['date'] = df_candles['begin'].dt.date
    
    df_news['publish_date'] = pd.to_datetime(df_news['publish_date'])
    df_news['date'] = df_news['publish_date'].dt.date
    
    # 2. Фильтрация новостей без тикера
    df_news_filtered = df_news[df_news['ticker'] != 'NONE'].copy()
    
    print(f"\n{dataset_name} - После фильтрации NONE:")
    print(f"Новости с тикерами: {len(df_news_filtered)} из {len(df_news)}")
    
    # 3. Агрегация новостей по дате и тикеру (если несколько новостей в день)
    df_news_agg = df_news_filtered.groupby(['ticker', 'date']).agg({
        'title': lambda x: ' | '.join(x),
        'publication': lambda x: ' | '.join(x),
        'publish_date': 'first'
    }).reset_index()
    
    print(f"{dataset_name} - После агрегации новостей: {len(df_news_agg)} уникальных (тикер+дата)")
    
    # 4. Merge по тикеру и дате
    df_merged = pd.merge(
        df_candles,
        df_news_agg,
        on=['ticker', 'date'],
        how='left'
    )
    
    # 5. Сортировка
    df_merged = df_merged.sort_values(['ticker', 'begin']).reset_index(drop=True)
    
    # 6. Статистика
    news_count = df_merged['title'].notna().sum()
    news_pct = (news_count / len(df_merged)) * 100
    
    print(f"{dataset_name} - Результат merge:")
    print(f"  Всего строк: {len(df_merged)}")
    print(f"  Строк с новостями: {news_count} ({news_pct:.2f}%)")
    print(f"  Уникальных тикеров: {df_merged['ticker'].nunique()}")
    
    return df_merged



In [ ]:
# ============================================
# ВЫПОЛНЯЕМ MERGE
# ============================================
train_merged = merge_candles_and_news(train, train_new, 'TRAIN')


In [ ]:
train = train_merged.copy()

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.to_csv("train_new.csv", index = False)

# тут train

In [2]:
import pandas as pd

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from transformers import pipeline
import warnings

# Игнорируем предупреждения для чистоты вывода
warnings.filterwarnings('ignore')

2025-10-04 23:49:27.718203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759621767.740780     106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759621767.747614     106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train = pd.read_csv('/kaggle/input/finam-train-new/train_new.csv')
train

,open,close,high,low,volume,begin,ticker,date,title,publication,publish_date
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT,2020-06-19,"Решение о допэмиссии не так негативно для ""Аэр...","По данным Reuters, правительство обсуждает воп...",2020-06-19 13:15:00
1,81.72,82.10,83.98,80.26,18502950,2020-06-22,AFLT,2020-06-22,"""Аэрофлот"" за пять месяцев 2020 года снизил ав...","За пять месяцев 2020 года группа ""Аэрофлот"" пе...",2020-06-22 11:10:53
2,82.04,81.20,82.48,80.40,16848930,2020-06-23,AFLT,2020-06-23,"Ослабление карантинных мер поможет ""Аэрофлоту""...","Результаты ""Аэрофлота"" оказались ожидаемо слаб...",2020-06-23 10:10:00
3,79.78,80.58,80.80,78.22,21559860,2020-06-25,AFLT,2020-06-25,NaN,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,2020-06-26,AFLT,2020-06-26,"ГОСА ""Аэрофлота"" пройдет 27 июля","Совет директоров ""Аэрофлота"" утвердил повестку...",2020-06-26 13:14:51
...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,2025-09-04,VTBR,2025-09-04,ВТБ размещает однодневные облигации на 100 млр...,Банк ВТБ размещает сегодня однодневные облигац...,2025-09-04 12:19:00
25938,75.85,75.50,76.13,74.94,40701093,2025-09-05,VTBR,2025-09-05,ВТБ построит детский лагерь и подарит его Камч...,Банк ВТБ построит на Камчатке детский лагерь «...,2025-09-05 09:38:36
25939,75.59,75.64,75.71,75.35,1681425,2025-09-06,VTBR,2025-09-06,NaN,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,2025-09-07,VTBR,2025-09-07,NaN,NaN,NaN


In [5]:
df = train.copy()

In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.drop(['publish_date', 'begin'], axis = 1)

In [7]:
df

,open,close,high,low,volume,ticker,date,title,publication
0,81.50,81.70,83.20,81.16,29755530,AFLT,2020-06-19,"Решение о допэмиссии не так негативно для ""Аэр...","По данным Reuters, правительство обсуждает воп..."
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,"""Аэрофлот"" за пять месяцев 2020 года снизил ав...","За пять месяцев 2020 года группа ""Аэрофлот"" пе..."
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,"Ослабление карантинных мер поможет ""Аэрофлоту""...","Результаты ""Аэрофлота"" оказались ожидаемо слаб..."
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,"ГОСА ""Аэрофлота"" пройдет 27 июля","Совет директоров ""Аэрофлота"" утвердил повестку..."
...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,ВТБ размещает однодневные облигации на 100 млр...,Банк ВТБ размещает сегодня однодневные облигац...
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,ВТБ построит детский лагерь и подарит его Камч...,Банк ВТБ построит на Камчатке детский лагерь «...
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,NaN,NaN


In [8]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ===== ИНИЦИАЛИЗАЦИЯ МОДЕЛЕЙ И ТОКЕНИЗАТОРА =====

print("Загрузка RuBERT для анализа тональности (финансовый русский)...")
model_name = "mxlcw/rubert-tiny2-russian-financial-sentiment"
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model_name,
    device=0 if torch.cuda.is_available() else -1,
    torch_dtype=torch.float16  # Добавляем FP16 для ускорения
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
MAX_TOKENS = 512

# Маппинг лейблов модели
LABEL_MAPPING = {
    'LABEL_0': 'neutral',
    'LABEL_1': 'positive',
    'LABEL_2': 'negative'
}

# ===== ОСНОВНАЯ ОБРАБОТКА =====

def main_processing(df):
    print("\n=== НАЧАЛО АНАЛИЗА ===\n")
    
    # Подготовка данных
    print("Подготовка данных...")
    df['title'].fillna("", inplace=True)
    df['publication'].fillna("", inplace=True)
    df['full_text'] = df['title'].astype(str) + ". " + df['publication'].astype(str)
    
    news_mask = df['full_text'].str.strip() != '.'
    print(f"Найдено {news_mask.sum()} записей с новостями.")
    if not news_mask.any():
        return df

    # Пакетный анализ тональности
    print("\n-> Анализ тональности (пакетная обработка)...")
    
    long_texts_map = {}
    short_texts_map = {}
    
    for idx, text in tqdm(df.loc[news_mask, 'full_text'].items(), desc="Токенизация"):
        if not text.strip():
            continue
        tokens = tokenizer.tokenize(text)
        if len(tokens) > MAX_TOKENS - 2:
            chunk_size = MAX_TOKENS - 2  # Максимальный размер чанка
            stride = 400  # Увеличили stride для уменьшения количества чанков ~в 2 раза
            text_chunks = []
            for i in range(0, len(tokens), stride):
                chunk_tokens = tokens[i:i + chunk_size]
                if chunk_tokens:
                    text_chunks.append(tokenizer.convert_tokens_to_string(chunk_tokens))
            if text_chunks:
                long_texts_map[idx] = text_chunks
        else:
            short_texts_map[idx] = text

    batch_short_indices = list(short_texts_map.keys())
    batch_short_texts = list(short_texts_map.values())
    
    batch_long_indices = []
    batch_long_texts_chunks = []
    for idx, chunks in long_texts_map.items():
        for chunk in chunks:
            batch_long_indices.append(idx)
            batch_long_texts_chunks.append(chunk)

    batch_size = 128 if torch.cuda.is_available() else 16  # Увеличили batch_size
    
    short_sentiments = []
    if batch_short_texts:
        print(f"Анализ {len(batch_short_texts)} коротких текстов...")
        short_sentiments = []
        with torch.no_grad():  # Добавили no_grad для экономии памяти и ускорения
            for i in tqdm(range(0, len(batch_short_texts), batch_size), desc="Анализ коротких текстов"):
                batch = batch_short_texts[i:i + batch_size]
                batch_results = sentiment_analyzer(batch, top_k=None, batch_size=batch_size)
                # Ремап лейблов
                for results in batch_results:
                    for r in results:
                        r['label'] = LABEL_MAPPING.get(r['label'], r['label'])
                short_sentiments.extend(batch_results)

    long_sentiments_chunks = []
    if batch_long_texts_chunks:
        print(f"Анализ {len(batch_long_texts_chunks)} чанков из {len(long_texts_map)} длинных текстов...")
        long_sentiments_chunks = []
        with torch.no_grad():
            for i in tqdm(range(0, len(batch_long_texts_chunks), batch_size), desc="Анализ чанков длинных текстов"):
                batch = batch_long_texts_chunks[i:i + batch_size]
                batch_results = sentiment_analyzer(batch, top_k=None, batch_size=batch_size)
                # Ремап лейблов
                for results in batch_results:
                    for r in results:
                        r['label'] = LABEL_MAPPING.get(r['label'], r['label'])
                long_sentiments_chunks.extend(batch_results)
    
    all_results = {}

    for i, index in enumerate(batch_short_indices):
        all_results[index] = short_sentiments[i]

    aggregated_long_sentiments = {}
    for i, original_index in enumerate(batch_long_indices):
        if original_index not in aggregated_long_sentiments:
            aggregated_long_sentiments[original_index] = []
        aggregated_long_sentiments[original_index].append(long_sentiments_chunks[i])

    for index, result_list_of_lists in aggregated_long_sentiments.items():
        max_sentiment_score = -1
        best_result = None
        for result_list in result_list_of_lists:
            probs = {r['label']: r['score'] for r in result_list}
            current_max = max(probs.get('positive', 0), probs.get('negative', 0))
            if current_max > max_sentiment_score:
                max_sentiment_score = current_max
                best_result = result_list
        if best_result:
            all_results[index] = best_result

    final_sentiment_data = []
    for index, results in all_results.items():
        probs = {r['label']: r['score'] for r in results}
        dominant = max(results, key=lambda x: x['score'])
        score = probs.get('positive', 0) - probs.get('negative', 0)
        final_sentiment_data.append({
            'index': index, 'sentiment_score': score, 'sentiment_label': dominant['label'],
            'positive_prob': probs.get('positive', 0), 'negative_prob': probs.get('negative', 0),
            'neutral_prob': probs.get('neutral', 0), 'confidence': dominant['score']
        })
    
    if final_sentiment_data:
        sentiment_df = pd.DataFrame(final_sentiment_data).set_index('index')
        df = df.join(sentiment_df)
    
    return df


df_processed = main_processing(df.copy())

df_processed.to_csv('news_analysis_complete.csv', index=False)

Загрузка RuBERT для анализа тональности (финансовый русский)...


Device set to use cuda:0



=== НАЧАЛО АНАЛИЗА ===

Подготовка данных...
Найдено 8034 записей с новостями.

-> Анализ тональности (пакетная обработка)...


Токенизация: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2148 > 2048). Running this sequence through the model will result in indexing errors
Токенизация: 8034it [00:31, 259.04it/s]


Анализ 4253 коротких текстов...


Анализ коротких текстов: 100%|██████████| 34/34 [00:08<00:00,  3.97it/s]


Анализ 17452 чанков из 3781 длинных текстов...


Анализ чанков длинных текстов: 100%|██████████| 137/137 [00:46<00:00,  2.92it/s]


In [9]:
df_processed = df_processed.drop(['title', 'publication', 'full_text'],axis = 1)

In [10]:
mapping = {
    'negative': -1,
    'neutral': 0,
    'positive': 1
}

df_processed['sentiment_label'] = df_processed['sentiment_label'].map(mapping)

In [11]:
df_processed

,open,close,high,low,volume,ticker,date,sentiment_score,sentiment_label,positive_prob,negative_prob,neutral_prob,confidence
0,81.50,81.70,83.20,81.16,29755530,AFLT,2020-06-19,-0.753348,-1.0,0.020257,0.773604,0.206139,0.773604
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,-0.808347,-1.0,0.072441,0.880788,0.046771,0.880788
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,0.656769,1.0,0.716765,0.059996,0.223239,0.716765
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,NaN,NaN,NaN,NaN,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,0.308057,0.0,0.319556,0.011499,0.668945,0.668945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,0.228645,0.0,0.312902,0.084258,0.602840,0.602840
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,0.721074,1.0,0.771358,0.050284,0.178359,0.771358
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,NaN,NaN,NaN,NaN,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import io


# 1. Предварительная обработка данных
print("Шаг 1: Предварительная обработка данных...")
df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed.sort_values(by=['ticker', 'date'], inplace=True)

sentiment_cols = ['sentiment_score', 'sentiment_label', 'positive_prob', 'negative_prob', 'neutral_prob', 'confidence']
df_processed[sentiment_cols] = df_processed.groupby('ticker')[sentiment_cols].ffill().bfill()
df_processed.dropna(subset=sentiment_cols, inplace=True)

df_processed['return'] = df_processed.groupby('ticker')['close'].pct_change()
df_processed.dropna(subset=['return'], inplace=True)

# 2. Определение признаков и констант
FEATURES = ['open', 'close', 'high', 'low', 'volume', 'return'] + sentiment_cols
TARGET = 'return'

Шаг 1: Предварительная обработка данных...


In [13]:
df_processed

,open,close,high,low,volume,ticker,date,sentiment_score,sentiment_label,positive_prob,negative_prob,neutral_prob,confidence,return
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,-0.808347,-1.0,0.072441,0.880788,0.046771,0.880788,0.004896
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.010962
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.007635
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,0.308057,0.0,0.319556,0.011499,0.668945,0.668945,-0.014892
5,79.00,82.90,83.00,77.44,37790740,AFLT,2020-06-29,-0.824868,-1.0,0.038297,0.863165,0.098538,0.863165,0.044344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,0.228645,0.0,0.312902,0.084258,0.602840,0.602840,0.008119
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,-0.003169
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.001854
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.000529


In [14]:
pip install TA-Lib -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tabm -q

Note: you may need to restart the kernel to use updated packages.


In [16]:
df_processed

,open,close,high,low,volume,ticker,date,sentiment_score,sentiment_label,positive_prob,negative_prob,neutral_prob,confidence,return
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,-0.808347,-1.0,0.072441,0.880788,0.046771,0.880788,0.004896
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.010962
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.007635
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,0.308057,0.0,0.319556,0.011499,0.668945,0.668945,-0.014892
5,79.00,82.90,83.00,77.44,37790740,AFLT,2020-06-29,-0.824868,-1.0,0.038297,0.863165,0.098538,0.863165,0.044344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,0.228645,0.0,0.312902,0.084258,0.602840,0.602840,0.008119
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,-0.003169
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.001854
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.000529


In [17]:
import numpy as np
import pandas as pd
import talib
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# ### НОВОЕ: Импортируем Dataset для создания кастомного загрузчика ###
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tabm import TabM # Ensure tabm library is installed (pip install tabm)
import warnings
import random

# Подавляем ненужные предупреждения для чистоты вывода
warnings.filterwarnings("ignore", category=FutureWarning)

# 1. ОПРЕДЕЛЕНИЕ УСТРОЙСТВА И КОНСТАНТ
# ==============================================================================
# --- Фиксация сидов для воспроизводимости ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
print(f"Все 'random seed' установлены на {SEED} для воспроизводимости.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

# --- Параметры модели ---
SEQUENCE_LENGTH = 60
PREDICTION_HORIZON_K = 20
EPOCHS = 100
BATCH_SIZE = 128 # Вы изменили с 256, оставляем 128
LEARNING_RATE = 1e-4
PATIENCE = 5

# --- Параметры BiLSTM ---
LSTM_HIDDEN_SIZE = 128
LSTM_NUM_LAYERS = 2

# --- Константы фичей ---
PATTERN_FUNCTIONS = talib.get_function_groups()['Pattern Recognition']
INDICATORS = {
    'SMA': [10, 20, 50],
    'EMA': [10, 20, 50],
    'RSI': [14],
    'MACD': [12, 26, 9],
    'ATR': [14]
}
TARGET = 'return'

# 2. ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
# ==============================================================================
# ПРЕДПОЛАГАЕТСЯ, ЧТО df_processed УЖЕ ЗАГРУЖЕН
# ... (код загрузки и первичной обработки данных остается без изменений) ...
df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed.sort_values(by=['ticker', 'date'], inplace=True)
df_processed['day_of_week'] = df_processed['date'].dt.dayofweek
sentiment_cols = ['sentiment_score', 'sentiment_label', 'positive_prob', 'negative_prob', 'neutral_prob', 'confidence']
df_processed[sentiment_cols] = df_processed.groupby('ticker')[sentiment_cols].ffill().bfill()
df_processed.dropna(subset=sentiment_cols, inplace=True)
df_processed['return'] = df_processed.groupby('ticker')['close'].pct_change()
df_processed.dropna(subset=['return'], inplace=True)

# 3. ГЕНЕРАЦИЯ ФИЧЕЙ
# ==============================================================================
# ... (код генерации фичей остается без изменений) ...
print("\nНачало генерации фичей для всех тикеров...")
all_tickers_data = []
for ticker, df_ticker in df_processed.groupby('ticker'):
    df_ticker = df_ticker.copy()
    for pattern_func in PATTERN_FUNCTIONS:
        try:
            indicator_func = getattr(talib, pattern_func)
            result = indicator_func(df_ticker['open'], df_ticker['high'], df_ticker['low'], df_ticker['close'])
            df_ticker[pattern_func] = result
        except Exception:
            df_ticker[pattern_func] = 0
    for name, params in INDICATORS.items():
        if name == 'MACD':
            macd, macdsignal, macdhist = talib.MACD(df_ticker['close'], fastperiod=params[0], slowperiod=params[1], signalperiod=params[2])
            df_ticker['MACD'] = macd
            df_ticker['MACD_signal'] = macdsignal
        elif name == 'ATR':
            df_ticker['ATR'] = talib.ATR(df_ticker['high'], df_ticker['low'], df_ticker['close'], timeperiod=params[0])
        else:
            for period in params:
                col_name = f'{name}_{period}'
                try:
                    indicator_func = getattr(talib, name)
                    df_ticker[col_name] = indicator_func(df_ticker['close'], timeperiod=period)
                except Exception:
                    df_ticker[col_name] = 0
    all_tickers_data.append(df_ticker)
df_full = pd.concat(all_tickers_data).sort_values(by=['ticker', 'date']).fillna(0)
print("Генерация фичей завершена.")

# --- ПОДСЧЕТ ФИЧЕЙ ---
df_full['ticker_code'] = df_full['ticker'].astype('category').cat.codes
base_features = ['open', 'close', 'high', 'low', 'volume', 'return']
time_features = ['day_of_week']
indicator_names = sorted([col for col in df_full.columns if any(name in col for name in INDICATORS)])
pattern_names = sorted([col for col in df_full.columns if col in PATTERN_FUNCTIONS])
ticker_code_feature = ['ticker_code']
FEATURES = sorted(list(set(base_features + time_features + sentiment_cols + indicator_names + pattern_names + ticker_code_feature)))
print(f"\n✅ Всего уникальных фичей: {len(FEATURES)}")
print(f"📈 Базовые признаки (base_features): {len(base_features)} признаков")
print(f"🗓️ Временные признаки (time_features): {len(time_features)} признак")
print(f"😊 Признаки настроений (sentiment_cols): {len(sentiment_cols)} признаков")
print(f"📊 Технические индикаторы (indicator_names): {len(indicator_names)} индикаторов")
print(f"📉 Паттерны (pattern_names): {len(pattern_names)} паттернов")
print(f"🔖 Код тикера (ticker_code_feature): {len(ticker_code_feature)} признак")


# 4. МАСШТАБИРОВАНИЕ И СОЗДАНИЕ DATASET'ов
# ==============================================================================
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_full[FEATURES])
scaled_df = pd.DataFrame(scaled_data, columns=FEATURES, index=df_full.index)
scaled_df[TARGET] = df_full[TARGET].values
scaled_df['ticker'] = df_full['ticker'].values


### НОВОЕ: Кастомный Dataset для эффективной работы с памятью ###
class TickerSequenceDataset(Dataset):
    def __init__(self, data, features_list, target_col, seq_length, pred_horizon):
        self.features_list = features_list
        self.target_col = target_col
        self.seq_length = seq_length
        self.pred_horizon = pred_horizon
        
        self.data_groups = {ticker: group.reset_index(drop=True) for ticker, group in data.groupby('ticker')}
        self.tickers = list(self.data_groups.keys())
        
        # Создаем "карту" всех возможных последовательностей без их материализации
        self.index_map = []
        for ticker_name in self.tickers:
            df_ticker = self.data_groups[ticker_name]
            max_start_index = len(df_ticker) - self.seq_length - self.pred_horizon + 1
            if max_start_index > 0:
                for i in range(max_start_index):
                    self.index_map.append((ticker_name, i))

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        # По индексу получаем тикер и стартовую позицию последовательности
        ticker_name, start_idx = self.index_map[idx]
        
        # Получаем данные только для этого тикера
        df_ticker = self.data_groups[ticker_name]
        
        # Вырезаем нужную последовательность и таргет
        sequence_end_idx = start_idx + self.seq_length
        target_idx = sequence_end_idx + self.pred_horizon - 1
        
        # Извлекаем данные как numpy-массивы
        features = df_ticker.loc[start_idx:sequence_end_idx-1, self.features_list].values
        target = df_ticker.loc[target_idx, self.target_col]
        
        # Конвертируем в тензоры
        return torch.tensor(features, dtype=torch.float32), torch.tensor(target, dtype=torch.float32).unsqueeze(0)


# ### НОВОЕ: Разделение данных по тикерам, а не по случайным последовательностям ###
# Это более правильный подход для временных рядов, чтобы избежать утечки данных
unique_tickers = scaled_df['ticker'].unique()
train_tickers, val_tickers = train_test_split(unique_tickers, test_size=0.2, random_state=SEED)

train_df = scaled_df[scaled_df['ticker'].isin(train_tickers)]
val_df = scaled_df[scaled_df['ticker'].isin(val_tickers)]

print(f"\nРазделили данные: {len(train_tickers)} тикеров для обучения, {len(val_tickers)} для валидации.")

# Создаем экземпляры кастомного Dataset
train_dataset = TickerSequenceDataset(train_df, FEATURES, TARGET, SEQUENCE_LENGTH, PREDICTION_HORIZON_K)
val_dataset = TickerSequenceDataset(val_df, FEATURES, TARGET, SEQUENCE_LENGTH, PREDICTION_HORIZON_K)

print(f"Количество последовательностей для обучения: {len(train_dataset)}")
print(f"Количество последовательностей для валидации: {len(val_dataset)}")

# Создаем DataLoader'ы, которые будут использовать наш кастомный Dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


# 5. ПОСТРОЕНИЕ И ОБУЧЕНИЕ МОДЕЛИ
# ==============================================================================
class BiLSTM_TabM(nn.Module):
    def __init__(self, num_features, lstm_hidden_size, lstm_num_layers, tabm_n_blocks, tabm_d_block, d_out):
        super(BiLSTM_TabM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            bidirectional=True
        )
        tabm_input_dim = SEQUENCE_LENGTH * lstm_hidden_size * 2
        self.tabm = TabM.make(
            n_num_features=tabm_input_dim,
            d_out=d_out,
            n_blocks=tabm_n_blocks,
            d_block=tabm_d_block
        )
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        batch_size = lstm_out.size(0)
        flattened_out = lstm_out.reshape(batch_size, -1)
        tabm_out = self.tabm(flattened_out)
        return tabm_out

# ### НОВОЕ: num_features теперь берется из списка FEATURES, а не из формы X_train ###
model = BiLSTM_TabM(
    num_features=len(FEATURES),
    lstm_hidden_size=LSTM_HIDDEN_SIZE,
    lstm_num_layers=LSTM_NUM_LAYERS,
    tabm_n_blocks=3,
    tabm_d_block=512,
    d_out=1
).to(device)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()
mae_criterion = nn.L1Loss()

print("\n--- Начало обучения гибридной модели BiLSTM-TabM ---")
best_val_mae = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    # ### НОВОЕ: Цикл обучения не меняется, но теперь он эффективно работает с памятью ###
    for batch_X, batch_y in train_loader:
        # Отправляем данные на нужное устройство
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        y_pred = model(batch_X).mean(dim=1) 
        loss = criterion(y_pred, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    val_maes = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            y_pred = model(batch_X).mean(dim=1) 
            val_mae = mae_criterion(y_pred, batch_y)
            val_maes.append(val_mae.item())

    avg_val_mae = np.mean(val_maes)
    print(f"Эпоха {epoch+1}/{EPOCHS}, Валидационная MAE: {avg_val_mae:.6f}")

    if avg_val_mae < best_val_mae:
        best_val_mae = avg_val_mae
        torch.save(model.state_dict(), 'bilstm_tabm_model_weights.pth')
        print(f"✅ Модель улучшилась. Веса сохранены. Лучшая MAE: {best_val_mae:.6f}")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Ранняя остановка на эпохе {epoch+1}.")
            break

# 6. ВЫВОД ИТОГОВОЙ МЕТРИКИ
# ==============================================================================

print("\n--- Обучение завершено ---")
print(f"🏆 Финальная лучшая MAE на валидационном наборе: {best_val_mae:.6f}")
print("💾 Веса лучшей модели сохранены в 'bilstm_tabm_model_weights.pth'")

Все 'random seed' установлены на 42 для воспроизводимости.
Используется устройство: cuda

Начало генерации фичей для всех тикеров...
Генерация фичей завершена.

✅ Всего уникальных фичей: 85
📈 Базовые признаки (base_features): 6 признаков
🗓️ Временные признаки (time_features): 1 признак
😊 Признаки настроений (sentiment_cols): 6 признаков
📊 Технические индикаторы (indicator_names): 11 индикаторов
📉 Паттерны (pattern_names): 61 паттернов
🔖 Код тикера (ticker_code_feature): 1 признак

Разделили данные: 15 тикеров для обучения, 4 для валидации.
Количество последовательностей для обучения: 19259
Количество последовательностей для валидации: 5144

--- Начало обучения гибридной модели BiLSTM-TabM ---
Эпоха 1/100, Валидационная MAE: 0.013665
✅ Модель улучшилась. Веса сохранены. Лучшая MAE: 0.013665
Эпоха 2/100, Валидационная MAE: 0.013661
✅ Модель улучшилась. Веса сохранены. Лучшая MAE: 0.013661
Эпоха 3/100, Валидационная MAE: 0.013642
✅ Модель улучшилась. Веса сохранены. Лучшая MAE: 0.013642
Э

In [18]:
# ==============================================================================
# 7. ГЕНЕРАЦИЯ ФАЙЛА САБМИТА
# ==============================================================================

print("\n--- Начало генерации файла сабмита ---")

# --- Загрузка лучшей модели ---
# Инициализируем модель с той же архитектурой
model = BiLSTM_TabM(
    num_features=len(FEATURES),
    lstm_hidden_size=LSTM_HIDDEN_SIZE,
    lstm_num_layers=LSTM_NUM_LAYERS,
    tabm_n_blocks=3,
    tabm_d_block=512,
    d_out=1
).to(device)


model.load_state_dict(torch.load('/kaggle/working/bilstm_tabm_model_weights.pth', map_location=device))
print("Веса лучшей модели успешно загружены.")

model.eval() # Переводим модель в режим оценки

# --- Подготовка к генерации прогнозов ---
submission_data = []
# Будем генерировать прогнозы для всех тикеров, которые есть в данных
all_unique_tickers = scaled_df['ticker'].unique()

print(f"Начинаем генерацию прогнозов для {len(all_unique_tickers)} тикеров...")

with torch.no_grad():
    for ticker in all_unique_tickers:
        ticker_df = scaled_df[scaled_df['ticker'] == ticker].copy()
        
        # Проверяем, достаточно ли данных для генерации полной последовательности прогнозов
        # Нам нужна как минимум (SEQUENCE_LENGTH + PREDICTION_HORIZON_K - 1) точка данных
        if len(ticker_df) < SEQUENCE_LENGTH + PREDICTION_HORIZON_K - 1:
            print(f"⚠️  Пропуск тикера {ticker}: недостаточно данных ({len(ticker_df)} точек) для генерации прогноза.")
            continue

        predictions = []
        
        # Генерируем 20 прогнозов (p1, p2, ..., p20)
        # Для каждого прогноза p_i сдвигаем входное окно данных
        for i in range(1, PREDICTION_HORIZON_K + 1):
            # Конечная точка среза для i-го прогноза
            # Чтобы спрогнозировать день T+i, модель, обученная на горизонте K,
            # должна получить на вход данные, заканчивающиеся в день T+i-K.
            end_idx = len(ticker_df) - (PREDICTION_HORIZON_K - i)
            start_idx = end_idx - SEQUENCE_LENGTH
            
            # Извлекаем последовательность фичей
            sequence = ticker_df.iloc[start_idx:end_idx][FEATURES].values
            
            # Преобразуем в тензор и добавляем batch-измерение
            X_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).to(device)
            
            # Делаем предсказание
            y_pred = model(X_tensor).mean(dim=1)
            
            # Сохраняем результат
            predictions.append(y_pred.item())

        # Создаем словарь для строки в итоговом файле
        row = {'ticker': ticker}
        row.update({f'p{i+1}': pred for i, pred in enumerate(predictions)})
        submission_data.append(row)

# --- Создание и сохранение DataFrame ---
if submission_data:
    submission_df = pd.DataFrame(submission_data)
    
    # Задаем правильный порядок столбцов
    column_order = ['ticker'] + [f'p{i}' for i in range(1, PREDICTION_HORIZON_K + 1)]
    submission_df = submission_df[column_order]
    
    # Сохраняем в CSV файл
    submission_df.to_csv('submission.csv', index=False)
    
    print("\n✅ Файл сабмита 'submission.csv' успешно сгенерирован.")
    print(f"   Количество тикеров в файле: {len(submission_df)}")
    print("   Пример сгенерированных данных:")
    print(submission_df.head())
else:
    print("\n⚠️ Не удалось сгенерировать данные для сабмита. Возможно, ни для одного тикера не хватило данных.")


--- Начало генерации файла сабмита ---
Веса лучшей модели успешно загружены.
Начинаем генерацию прогнозов для 19 тикеров...

✅ Файл сабмита 'submission.csv' успешно сгенерирован.
   Количество тикеров в файле: 19
   Пример сгенерированных данных:
  ticker        p1        p2        p3        p4        p5        p6  \
0   AFLT  0.001220  0.001108  0.001043  0.000885  0.000823  0.000784   
1   ALRS  0.000923  0.001014  0.001108  0.001181  0.001364  0.001599   
2   CHMF  0.000918  0.000900  0.000822  0.000762  0.000856  0.000898   
3   GAZP  0.000292  0.000926  0.001333  0.001539  0.001732  0.001910   
4   GMKN  0.001297  0.001368  0.001558  0.001676  0.001696  0.001683   

         p7        p8        p9  ...       p11       p12       p13       p14  \
0  0.000789  0.000843  0.000944  ...  0.000787  0.000882  0.001057  0.001132   
1  0.001829  0.002045  0.002155  ...  0.002030  0.002016  0.001796  0.001561   
2  0.000946  0.000964  0.000926  ...  0.000772  0.000661  0.000506  0.000446   